# Import the necessary libraries

In [1]:
!pip uninstall -y spacy

Found existing installation: spacy 2.3.9
Uninstalling spacy-2.3.9:
  Successfully uninstalled spacy-2.3.9


In [2]:
!python -m pip install spacy~=2.0

  Using cached spacy-2.3.9-cp39-cp39-win_amd64.whl.metadata (16 kB)
Using cached spacy-2.3.9-cp39-cp39-win_amd64.whl (9.1 MB)


In [3]:
!pip list

Package                           Version
--------------------------------- ------------
absl-py                           2.1.0
aiobotocore                       2.7.0
aiohttp                           3.9.3
aioitertools                      0.7.1
aiosignal                         1.2.0
alabaster                         0.7.12
altair                            5.0.1
anyio                             4.2.0
appdirs                           1.4.4
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
arrow                             1.2.3
astroid                           2.14.2
astropy                           5.3.4
asttokens                         2.0.5
astunparse                        1.6.3
async-lru                         2.0.4
async-timeout                     4.0.3
atomicwrites                      1.4.0
attrs                             23.1.0
Automat                           20.2.0
autopep8                          1.6.0
Babel                    

In [4]:
!pip uninstall -y spacy-transformers

In [5]:
import spacy
import pickle
import random


# Load the training data

In [6]:
train_data = pickle.load(open('C:/Users/KIIT/Downloads/Minor_project/ResumeParsing code/train_data.pkl', 'rb'))
train_data[0]

('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [7]:
nlp = spacy.blank('en')
def train_model(train_data, test_data):
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last = True)

    for _, annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(1000):
            print("Starting iteration " + str(itn))
            random.shuffle(train_data)
            losses = {}
            for text, annotations in train_data:
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorize data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                except Exception as e:
                    pass

            # Compute accuracy on test dataset
            correct = 0
            total = 0
            for text, annotations in test_data:
                doc = nlp(text)
                for ent in doc.ents:
                    if (ent.start_char, ent.end_char, ent.label_) in annotations['entities']:
                        correct += 1
                    total += 1
            accuracy = correct / total
            print(f"Accuracy after iteration {itn}: {accuracy:.3f}")


In [8]:
train_model(train_data, train_data)

Starting iteration 0
Accuracy after iteration 0: 0.757
Starting iteration 1
Accuracy after iteration 1: 0.502
Starting iteration 2
Accuracy after iteration 2: 0.440
Starting iteration 3
Accuracy after iteration 3: 0.491
Starting iteration 4
Accuracy after iteration 4: 0.530
Starting iteration 5
Accuracy after iteration 5: 0.546
Starting iteration 6
Accuracy after iteration 6: 0.504
Starting iteration 7
Accuracy after iteration 7: 0.546
Starting iteration 8
Accuracy after iteration 8: 0.579
Starting iteration 9
Accuracy after iteration 9: 0.549
Starting iteration 10
Accuracy after iteration 10: 0.576
Starting iteration 11
Accuracy after iteration 11: 0.614
Starting iteration 12
Accuracy after iteration 12: 0.618
Starting iteration 13
Accuracy after iteration 13: 0.646
Starting iteration 14
Accuracy after iteration 14: 0.634
Starting iteration 15
Accuracy after iteration 15: 0.645
Starting iteration 16
Accuracy after iteration 16: 0.649
Starting iteration 17
Accuracy after iteration 17: 

# Save the model for further use

In [9]:
nlp.to_disk('nlp_model')

# Load the saved model into nlp_model

In [10]:
nlp_model = spacy.load('nlp_model')

# First resume from our training data

In [11]:
train_data[0]

('Bike Rally Chief Coordinator of LEAR  Palghat, Kerala - Email me on Indeed: indeed.com/r/Bike-Rally/e00d408e91e83868  Education  WORK EXPERIENCE  Chief Coordinator of LEAR  Microsoft -  June 2011 to July 2013  under Office Kerala State Board  Edius tally  Proshow gold Key roles held Photoshop  Chief Coordinator of LEAR It is a well equipped training team under the mentorship of JCI International member and HR international Trainer Dr. Thomas K George, Chairman of LEAD College of Management, Dhoni.  Get in Touch Key Organizer of Le Adventure, I LEAD and LEADing bands These programs were conducted by LEAD College of Management.  Special Police Officer during Panchayat elections  Microsoft -  June 2011 to July 2013  ambras@lead.ac.in  EDUCATION  MBA in marketing-tourism  Calicut University -  Calicut, Kerala  2016 to 2018  https://www.indeed.com/r/Bike-Rally/e00d408e91e83868?isid=rex-download&ikw=download-top&co=IN',
 {'entities': [(826, 830, 'Graduation Year'),
   (779, 797, 'College N

# Pass the resume to the model and see results

In [12]:
doc = nlp_model(train_data[0][0])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Bike Rally
DESIGNATION                   - Chief Coordinator of LEAR
LOCATION                      - Palghat,
EMAIL ADDRESS                 - indeed.com/r/Bike-Rally/e00d408e91e83868
DESIGNATION                   - Chief Coordinator of LEAR
COMPANIES WORKED AT           - Microsoft
DESIGNATION                   - Chief Coordinator of LEAR
COMPANIES WORKED AT           - Microsoft
DEGREE                        - MBA in marketing-tourism
COLLEGE NAME                  - Calicut University
GRADUATION YEAR               - 2018


In [15]:
!pip install PyMuPDF

In [16]:
import sys, fitz
fname = 'Alice Clark CV.pdf'
doc = fitz.open(fname)
text = ""
for Page in doc:
    text = text + str(Page.get_text())

tx = " ".join(text.split('\n'))
print(tx)

Alice Clark  AI / Machine Learning    Delhi, India Email me on Indeed  •  20+ years of experience in data handling, design, and development  •  Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to  data warehousing and business intelligence  •  Database: Experience in database designing, scalability, back-up and recovery, writing and  optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes.  Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure,  Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake  analytics(U-SQL)  Willing to relocate anywhere    WORK EXPERIENCE  Software Engineer  Microsoft – Bangalore, Karnataka  January 2000 to Present  1. Microsoft Rewards Live dashboards:  Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping  online. Microsoft Rewards members can earn points when searching with Bing, bro

In [17]:
doc = nlp_model(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Alice Clark
LOCATION                      - Delhi
COMPANIES WORKED AT           - Microsoft
DESIGNATION                   - Software Engineer
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COLLEGE NAME                  - Indian Institute of Technology – Mumbai
GRADUATION YEAR               - 2001
